# Experiment: _Combined text-/graph features vs. text-only features_

In [ ]:
from notebook_prelude import *

In [ ]:
NA_VAL = '-'
EXPERIMENT_NAME = 'experiment_combined'
experiment_data = experiment_helper.get_experiment_config_for(EXPERIMENT_NAME)
param_grid = experiment_data['params_per_type']

#, EXPERIMENT_NAME + '_same_label', EXPERIMENT_NAME + '_with_splitted'
df = results_helper.get_experiments_by_names([EXPERIMENT_NAME])
df = df.fillna(NA_VAL)
pipeline_helper.remove_complex_types(pipeline_helper.flatten_nested_params(param_grid))

In [ ]:
VECTORIZER_TFIDF = 'TfidfVectorizer'
VECTORIZER_COUNT = 'CountVectorizer'

## Result table

In [ ]:
for c in df.columns: print(c)

In [ ]:
df[np.isin(df.type, [TYPE_CONCEPT_MAP, TYPE_COOCCURRENCE, 'text'])].groupby(['dataset', 'features__fast_wl_pipeline__feature_extraction__graph_preprocessing', 'text__vectorizer', 'combined', 'type']).mean_test_f1_macro.max().to_frame().unstack().unstack()#.unstack()

In [ ]:
for file, params in sorted(experiment_helper.get_all_param_grid_config_files().items(), key=lambda x: x[0]):
    file = file.split('/')[-1]
    print(file)
    for task_name, params_ in params['params_per_type'].items():
        if 'classifier__C' not in params_: continue
        C = params_['classifier__C']
        if not C: continue
        print('\t{:40} {}'.format(task_name, C))
    #print(file)

## Significance tests

In [ ]:
[x for x in results_helper.get_predictions_files() if EXPERIMENT_NAME in x and filename_utils.get_dataset_from_filename(x) == 'ng20' and 'graph_combined' in x]

In [ ]:
from utils import significance_test_utils

NUM_TRAILS = 10000
metric = significance_test_utils.f1

combinations = [
    # TfidfVectorizer
    (
        # Combined
        'result___experiment_combined__ng20__graph_combined__dataset_graph_concept_map_ng20-v3.npy',
        
        # Text-only
        'result___experiment_combined__ng20__text.npy',
    )
]

filenames = []
for a, b in combinations:
    filenames.append(a)
    filenames.append(b)

data = collections.defaultdict(lambda: [])

predictions = {k.split('/')[-1]: v['results']['results'] for k, v in results_helper.get_predictions(filenames=filenames)}
for filenames in combinations:
    assert np.all([x in predictions for x in filenames])
    models = [predictions[x] for x in filenames]
    keys = ['Y_real', 'Y_pred', 'X_test']
    assert np.all([len(models[0][key]) == len(models[1][key]) for key in keys])
    y_true = models[0]['Y_real']
    y_preds = [model['Y_pred'] for model in models]
    y_pred_a, y_pred_b = y_preds
    metric_real = [metric(y_true, y_pred) for y_pred in y_preds]
    diff_global = metric_real[0] - metric_real[1]
    
    metrics = significance_test_utils.randomization_test(y_true, y_pred_a, y_pred_b, metric=significance_test_utils.f1, num_trails=NUM_TRAILS)
    diffs = metrics[:, 0] - metrics[:, 1]
    confidence = significance_test_utils.get_confidence(diff_global, diffs, num_trails=NUM_TRAILS, one_tail=True)

    data['num_docs'].append(len(y_pred_a))
    data['filename_a'].append(filenames[0])
    data['filename_b'].append(filenames[1])
    data['confidence'].append(confidence)
    data['diffs'].append(diffs)
    data['diff_global'].append(diff_global)
    data['metric_a'].append(metric_real[0])
    data['metric_b'].append(metric_real[1])
    data['num_trails'].append(NUM_TRAILS)

In [ ]:
df_ = pd.DataFrame(data).set_index(['filename_a', 'filename_b'])
df_[[x for x in df_.columns if x != 'diffs']].T

In [ ]:
for (f_a, f_b), df__ in df_.iterrows():
    diffs = df__.diffs
    fig, ax = plt.subplots(figsize=(10, 3.4))
    significance_test_utils.plot_randomization_test_distribution_(diffs, df__.diff_global, num_trails=df__.num_trails, p=df__.confidence, metric_name='f1 macro', ax=ax)

In [ ]:
c = collections.Counter()
for idx, (a, b) in enumerate(zip(*y_preds)):
    c['same' if a == b else 'not_same'] += 1
c